In [1]:
# モジュールのインポート
import re
import csv
import time
import glob
import pandas as pd
import matplotlib.pyplot as plt
import MeCab
import random

In [ ]:
# 極性辞書の読み込み
pn_df = pd.read_csv('pn_dic.csv',
                    sep=',',
                    encoding='shift-jis',
                    index_col = 0,                 
                   )

# dict型に変換
word_list = list(pn['Word'])
pn_list = list(pn['PN'])
pn_dict = dict(zip(word_list, pn_list))

In [12]:
# MeCabインスタンス作成
m = MeCab.Tagger('')  # 指定しなければIPA辞書

# -----テキストを形態素解析して辞書のリストを返す関数----- #
def get_diclist(text):
    parsed = m.parse(text)      # 形態素解析結果（改行を含む文字列として得られる）
    lines = parsed.split('\n')  # 解析結果を1行（1語）ごとに分けてリストにする
    lines = lines[0:-2]         # 後ろ2行は不要なので削除
    diclist = []
    for word in lines:
        l = re.split('\t|,',word)  # 各行はタブとカンマで区切られてるので
        d = {'Surface':l[0], 'POS1':l[1], 'POS2':l[2], 'BaseForm':l[7]}
        diclist.append(d)
    return(diclist)

           Word        PN
0          ２，３日  0.000000
1           10%  0.000000
2          100%  0.000000
3           25%  0.000000
4           80%  0.000000
5          １０カ月  0.000000
6          １０時間  0.000000
7          １０数年  0.000000
8         １０年以上  0.000000
9         １０年近く  0.000000
10         １１ヶ月  0.000000
11          １１年  0.000000
12          １２年  0.000000
13          １３年  0.000000
14          １４年  0.000000
15          １５秒  0.000000
16          １６年  0.000000
17          １７年  0.000000
18          １９年  0.000000
19         １バイト  0.000000
20           １位  0.300000
21           １割  0.000000
22        １時間以上  0.000000
23        １時間以内  0.000000
24         １時間半  0.000000
25        １周年記念  0.300000
26        １週間以上  0.000000
27           １勝  0.300000
28           １体  0.000000
29           １通  0.000000
...         ...       ...
18557     愚の骨頂の -0.900170
18558   融通の利かない -0.907250
18559      根性曲り -0.915699
18560      根性悪の -0.915699
18561     根性悪です -0.915699
18562    好きものです -0.932040
18563     愛想

In [ ]:
# 形態素解析結果の単語ごとdictデータにPN値を追加する関数
def add_pnvalue(diclist_old):
    diclist_new = []
    for word in diclist_old:
        base = word['BaseForm']        # 個々の辞書から基本形を取得
        if base in pn_dict:
            pn = float(pn_dict[base])  # 中身の型があれなので
        else:
            pn = 'notfound'            # その語がPN Tableになかった場合
        word['PN'] = pn
        diclist_new.append(word)
    return(diclist_new)

In [ ]:
# テスト
test_text = 'STAP細胞はあります。'
dl_test = get_diclist(test_text)
dl_test = add_pnvalue(dl_test)
print(dl_test)

In [ ]:
# 各テキストのPN平均値をとる関数
def get_pnmean(diclist):
    pn_list = []
    for word in diclist:
        pn = word['PN']
        if pn != 'notfound':
            pn_list.append(pn)  # notfoundだった場合は追加もしない            
    if len(pn_list) > 0:        # 「全部notfound」じゃなければ
        pnmean = mean(pn_list)
    else:
        pnmean = 0              # 全部notfoundならゼロにする
    return(pnmean)

In [ ]:
# pn値のリストを作成（時間を測る）
start_time = time.time()               ### ▼時間計測▼ ###
pnmeans_list = []
for tw in tw_df['text']:
    dl_old = get_diclist(tw)
    dl_new = add_pnvalue(dl_old)
    pnmean = get_pnmean(dl_new)
    pnmeans_list.append(pnmean)
print(time.time() - start_time)        ### ▲時間計測▲ ###

In [ ]:
# 本文テキストから改行を除いておく
text_list = list(tw_df['text'])
for i in range(len(text_list)):
    text_list[i] = text_list[i].replace('\n', ' ')


In [ ]:
# 記事タイトル、PN値を格納したデータフレームを作成
aura_df = pd.DataFrame({'tweet_id':tw_df['tweet_id'],
                        'text':text_list,
                        'PN':pnmeans_list,
                       },
                       columns=['tweet_id', 'text', 'PN']
                      )

# PN値の昇順でソート
aura_df = aura_df.sort_values(by='PN', ascending=True)

# CSVを出力（ExcelでみたいならUTF8ではなくShift-JISを指定すべき）
aura_df.to_csv('aura.csv',\
                index=None,\
                encoding='utf-8',\
                quoting=csv.QUOTE_NONNUMERIC\
               )